In [ ]:
import PyKEP as pk
import pygmo as pg
import pygmo_plugins_nonfree as pg7
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

In [ ]:
# algorithm
uda = pg7.snopt7(True, "/usr/lib/libsnopt7_c.so")
uda.set_numeric_option("Major optimality tolerance", 1e-6)
uda.set_numeric_option("Major feasibility tolerance", 1e-10)
uda.set_integer_option("Major iterations limit", 100)
algo = pg.algorithm(uda)

In [ ]:
# constants
pi = 3.1451592

# planets
p0 = pk.planet.jpl_lp("earth")
pf = pk.planet.jpl_lp("mars")

# Keplerian elements
el0 = np.array(p0.osculating_elements(pk.epoch(0)))
elf = np.array(pf.osculating_elements(pk.epoch(0)))

# spacecraft
mass   = 1000
thrust = 0.3
isp    = 2500

# tolerances
atol = 1e-12
rtol = 1e-12

# flight duration bounds
Tlb = 100
Tub = 2000

# eccentric anomoly bounds
Mlb = -4*pi
Mub = 4*pi

In [ ]:
# problem
udp = pk.trajopt.indirect_or2or(
    el0, elf, mass, thrust, isp, atol, rtol,
    Tlb, Tub, Mlb, Mub, Mlb, Mub, alpha=0, bound=False
)
prob = pg.problem(udp)
prob.c_tol = [1e-6]*udp.get_nec()

In [ ]:
# population
pop = pg.population(prob, 0)
z = np.hstack(([np.random.uniform(Tlb, Tub)], np.random.randn(9)))
pop.push_back(z)
prob.fitness(pop.champion_x)

In [ ]:
while not prob.feasibility_x(pop.champion_x):
    pop = pg.population(prob, 0)
    z = np.hstack(([np.random.uniform(Tlb, Tub)], np.random.randn(9)))
    pop.push_back(z)
    pop = algo.evolve(pop)

In [ ]:
pop.champion_x

In [ ]:
udp.plot_traj(pop.champion_x)
udp.plot_control(pop.champion_x)

In [ ]:
np.save("../../npy/indirect_or2or1.npy", pop.champion_x)

In [ ]:
pop = pg.population(prob, 0)
pop.push_back(np.load("../../npy/indirect_or2or1.npy"))
pop = algo.evolve(pop)

In [ ]:
udp.leg.plot("t", "lm", xlabel=True, ylabel=True)

In [ ]:
udp.leg.get_states()[-1, 7]